In [22]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd

try:
    import GOSTRocks.rasterMisc as rMisc    
    from GOSTRocks.misc import tPrint
except:
    print("Cannot find GOSTRocks")


Cannot find GOST_Urban


In [9]:
ghs_smod = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_SMOD/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0.tif"
ghs_pop =  "/home/public/Data/GLOBAL/Population/GHS/GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0.tif"
urbanVals = [11,12,13,21,22,23,30]

urbanR = rasterio.open(ghs_smod)
popR = rasterio.open(ghs_pop)

In [3]:
urbanR.shape

(18000, 36082)

In [4]:
popR.shape

(18000, 36082)

In [26]:
extents_shapefile = "/home/wb411133/data/Projects/SD_FLAGSHIP/Data/Global_fishnet_clip.shp"
inD = gpd.read_file(extents_shapefile)
if inD.crs != urbanR.crs:
    inD = inD.to_crs(urbanR.crs)

In [27]:
# calculate area of each class in extents
res_urban = rMisc.zonalStats(inD, urbanR, rastType='C', unqVals=urbanVals)
res_urban = pd.DataFrame(res_urban, columns=['c_%s' % x for x in urbanVals])
res_urban.head()

,c_11,c_12,c_13,c_21,c_22,c_23,c_30
0,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1


In [61]:
try:
    res_urban.drop(['major_lc_area'], axis=1, inplace=True)
except:
    pass

def get_max_column(x):
    x = x.astype(float)
    try:        
        return(x.idxmax())
    except:
        return('-1')

res_urban['major_lc_area'] = res_urban.apply(get_max_column, axis=1)

In [62]:
res_urban['major_lc_area'].value_counts()

c_11    94994
c_12      515
c_21      207
c_30      175
c_13       60
c_23       12
c_22        7
Name: major_lc_area, dtype: int64

In [18]:
urban_data = urbanR.read()
pop_data = popR.read()

In [29]:
# calculate population in each urban class
allRes = inD
for urban_val in urbanVals:
    tPrint(urban_val)
    curR = urban_data == urban_val
    curPop = curR * pop_data
    with rMisc.create_rasterio_inmemory(popR.profile, curPop) as tempPopR:
        resPop = rMisc.zonalStats(inD, tempPopR, minVal=0)
        resPop = pd.DataFrame(resPop, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
        inD['Pop_%s' % urban_val] = resPop['SUM']
        
        

22:27:41	11
22:30:18	12
22:33:06	13
22:35:53	21
22:38:40	22
22:41:27	23
22:44:15	30


In [72]:
try:
    inD.drop(['major_lc_pop'], axis=1, inplace=True)
except:
    pass

def is_unique(s):
    a = s.to_numpy() # s.values (pandas<0.24)
    return (a[0] == a).all()

def get_max_column(x):
    x = x.astype(float)
    if is_unique(x):
        return('-1')
    try:        
        return(x.idxmax())
    except:
        return('-1')

inD['major_lc_pop'] = inD.iloc[:,-7:].apply(get_max_column, axis=1)

In [73]:
inD['major_lc_pop'].value_counts()

Pop_12    32795
-1        26894
Pop_11    11846
Pop_13     9014
Pop_23     7105
Pop_30     6080
Pop_22     1276
Pop_21      960
Name: major_lc_pop, dtype: int64

In [74]:
inD.join(res_urban).to_csv(extents_shapefile.replace(".shp", "_GHS_URBAN.csv"))

In [75]:
inD.crs

<Projected CRS: ESRI:54009>
Name: World_Mollweide
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Mollweide
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [76]:
inD.head()

,OBJECTID,Id,Lon,Lat,Shape_Leng,Shape_Area,geometry,Pop_11,Pop_12,Pop_13,Pop_21,Pop_22,Pop_23,Pop_30,major_lc_pop
0,1,0,-179.75,-89.75,2.0,0.25,"POLYGON ((-0.000 -9020047.848, -0.000 -9020047...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1
1,2,0,-179.25,-89.75,2.0,0.25,"POLYGON ((-0.000 -9020047.848, -0.000 -9020047...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1
2,3,0,-178.75,-89.75,2.0,0.25,"POLYGON ((-0.000 -9020047.848, -0.000 -9020047...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1
3,4,0,-178.25,-89.75,2.0,0.25,"POLYGON ((-0.000 -9020047.848, -0.000 -9020047...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1
4,5,0,-177.75,-89.75,2.0,0.25,"POLYGON ((-0.000 -9020047.848, -0.000 -9020047...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1


In [77]:
inD.columns

Index(['OBJECTID', 'Id', 'Lon', 'Lat', 'Shape_Leng', 'Shape_Area', 'geometry',
       'Pop_11', 'Pop_12', 'Pop_13', 'Pop_21', 'Pop_22', 'Pop_23', 'Pop_30',
       'major_lc_pop'],
      dtype='object')